# Bringing Functional Programming into an Imperative World.
# Maybe.


<img src="Pell_Contact.png">


<p style="font-size: 24px">You can follow along at:</p>
<p style="font-size: 24px">`https://github.com/gignosko/DjangoCon_2017`</p>

# Why FP?


* Expressive
* Efficient
* Safe-er
* Easier to work with concurrent and parallel programming


## What is Functional Programming?

Functional Programming is a style of programming that utilizes pure functions (functions with no side effects) to transform data.

At it's heart, though, functional programming is just a different way to think about programming and along with that, it helps to have a new set of tools. That's what we'll look at now.

# Higher order functions


* Functions get treated like first class citizens
    * Passed into other functions as parameters
    * Returned from functions as values
    * Stored in variables

### Passing in a function

In [1]:
def call_db(fn):
    print("Initializing database connection")
    curs = "cursor"
    return fn(curs)
    
def query_db(curs):
    results = "Query db using {}".format(curs)
    return results

def insert_into_db(curs):
    new_row_number = "Insert item into db using {}".format(curs)
    return new_row_number


In [2]:
result = call_db(query_db)
print(result)

Initializing database connection
Query db using cursor


In [3]:
new_row = call_db(insert_into_db)
print(new_row)

Initializing database connection
Insert item into db using cursor


### Returning a function

In [4]:
def complex_s3_fn(s):
    print("Hold up, I'm doing something complex on AWS with your {}". format(s))
    
def complex_sql_fn(s):
    print("Hold up, I'm doing something complex with your {}". format(s))
    
def give_me_a_fn(source):
    if source == "s3":
        return complex_s3_fn
    else:
        return complex_sql_fn
    

In [5]:
source = "s3"
new_fn = give_me_a_fn(source)
new_fn("S3 object")

Hold up, I'm doing something complex on AWS with your S3 object


In [6]:
source = "sql"
new_fn = give_me_a_fn(source)
new_fn("MySQL object")

Hold up, I'm doing something complex with your MySQL object


In [7]:
source = "sql"
give_me_a_fn(source)("Postgres Object")

Hold up, I'm doing something complex with your Postgres Object


### Decorators

In [8]:
def decorate_it(fn):
    def wrapper(*args):
        print("Let's wrap this up")
        fn(*args)
        print("Consider yourself decorated")
    return wrapper

In [9]:
@decorate_it
def add_it(x, y):
    print("It's gonna equal: ", x + y)

In [10]:
add_it(3, 5)

Let's wrap this up
It's gonna equal:  8
Consider yourself decorated


# Immutable Data

* The structure itself never gets changed
* New way of thinking about "updating" your data structure
    * every update returns a new data structure
* Safety
    * since each update returns a new data structure, you can safely pass them around
* Historically inefficient, but that's changing
    * [Persistent](https://en.wikipedia.org/wiki/Persistent_data_structure) data structures are beginning to make thier way into languages

### The problem with mutable data structures
* You can't be sure that your data won't change out from under you

In [11]:
list_1 = [1, 2, 3, 4]
list_2 = list_1
list1 = list_1.append(5)

print(list_1)
print(list_2)
print(list_1 is list_2)

[1, 2, 3, 4, 5]
[1, 2, 3, 4, 5]
True


In [13]:
def sum_list(lst):
    sum = 0
    for l in lst:
        #lst.append(l + 1)
        sum = sum + l
        if len(lst) > 20:
            break
    return sum

print(sum_list([1, 2, 3]))

81


### Immutable data structures help avoid this problem
* Python has a few immutable types, the most common is the tuple

In [14]:
tup_1 = 1, 2, "me"
tup_2 = tup_1
print(tup_1 is tup_2)

# adding to a tuple is slow because it has to copy the entire tuple
tup_1 = (4,) + tup_1

print(tup_1 is tup_2)
print(tup_1)
print(tup_2)

True
False
(4, 1, 2, 'me')
(1, 2, 'me')


### You can fake an immutable data structure using deepcopy
* But it's still slow

In [15]:
from copy import deepcopy
list_1 = [1, 2, 3, 4]
list_2 = deepcopy(list_1)

list_1.append(5)

print(list_1)
print(list_2)

[1, 2, 3, 4, 5]
[1, 2, 3, 4]


# Recursion

* Calling a function from inside itself. 

* Recursion takes the place of loops in many functional languages

* Every time you call, you add to the stack

* Tail call optimizations eliminate this if you design your function properly
    * But, python doesn't have TCO


### Let's look at recursion

In [16]:
def sum_list(l, s=0):
    if not l:
        return s
    s = s + l.pop()
    return sum_list(l, s)

In [17]:
l = [1, 2, 3]
q = sum_list(l)
print(q)

6


### The problem with recursion in python

In [18]:
l = list(range(1000))
q = sum_list(l)
print(q)

RecursionError: maximum recursion depth exceeded

# Map, Filter, Reduce, Lambdas

## Lambdas

* Anonymous functions
    * This just means they don't have a name

In [19]:
sum = lambda x, y: x + y

sum(2,3)

5

## Filter

* Takes a filtering function and a collection to operate on
* Applies the function to each item in the list, returning an iterator over the new list
* The filtering function needs to evaluate to True or False
    * If the function returns True on an item, that item goes into the new list
    * If it returns False, that item is excluded from the new list

In [20]:
list_1 = [1, 2, 3, 4]

In [21]:
list_2_iter =filter(lambda x: x % 2 == 0, list_1)

list_2 = list(list_2_iter)
print(list_1)
print(list_2)

[1, 2, 3, 4]
[2, 4]


## Map

* Takes a mapping function and a collection to operate on
* Applies the function to each item in the list, returning an iterator over the new list

In [22]:
list_3_iter = map(lambda x: x *2, list_1)
list_3 = list(list_3_iter)
print(list_3)

[2, 4, 6, 8]


## Reduce

* Takes a reducing function and a collection to reduce and an optional initial value
* The function needs to take 2 parameters and return some combination of the two in the same form as the parameters
* If there's no initial value, it passes the first two collection items into the function

In [23]:
from functools import reduce
list_1 = [1, 2, 3, 4]
reduce(lambda x, y: x + y, list_1)

10

In [27]:
big_list = list(range(1000))
reduce(lambda x, y: x + y, big_list)

499500

In [ ]:
dna = ["AAACTCTGGT", "AACTGGTC", "CCCTGTGT"]

a_count = reduce(lambda a, x: a + x.count("A"), dna, 0)
a_count

In [28]:
def combined_age(person1, person2):
    age1 = person1[1]
    age2 = person2[1]
    c_age = ["combined", age1 + age2]
    return c_age

users = {1: ["Rob", 25], 2: ["Jon", 21], 3: ["Arya", 15], 4: ["Sansa", 18], 5: ["Bran", 13]}

combined = reduce(combined_age, users.values())
print(combined)

['combined', 92]


## you can mimic map and filter in list comprehensions

### map

In [29]:
list_2 =[(lambda x: x*x)(x) for x in list_1]
print(list_2)

[1, 4, 9, 16]


### or if the lambda is too ugly

In [30]:
def square(x):
    return x*x

[square(x) for x in list_1]


[1, 4, 9, 16]

### filter


In [31]:
[x for x in list_1 if x % 2 == 0]

[2, 4]

# Should we bring FP into our python code?

* Of course!
    * FP can be less verbose
    * FP  can be more efficient
    * FP can help you think through problems in ways that are more intuitive
    
* But, be aware of the risks and shortcomings
    * Working with immutable data can be slower
    * Recursion can blow the stack up pretty quickly
    * Functional code looks weird